In [ ]:
#This source code converts real-time incoming data into a count-based sliding window method.
#The converted dataset is also used as input to the learned model to predict behavior in real time.
#이 소스코드는 실시간으로 들어오는 데이터를 카운트 기반 슬라이딩 윈도우 방식으로 전환합니다. 
#또한 전환된 데이터셋은 학습된 모델의 인풋으로 사용되어 실시간으로 행동을 예측합니다.

import socket
import sys
import csv
import datetime
from datetime import timedelta
import time
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
import statistics

loaded_model = pickle.load(open('C:/Users/Shin/Documents/testbeddata/RF_sc.sav', 'rb'))

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = ('192.168.0.108', 9999)
print('connecting to {} port {}'.format(*server_address))
sock.connect(server_address)

windowlist = []
timelist = []
def blankreturn(a):
    if len(a) == 7:
        blankadd = a
    elif len(a) == 6:
        blankadd = a + " "
    elif len(a) == 5:
        blankadd = a + "  "
    elif len(a) == 4:
        blankadd = a + "   "
    elif len(a) == 3:
        blankadd = a + "    "
    return blankadd

In [ ]:
while True:
    data = sock.recv(16)
    if data and data.decode('utf-8').find('On')>0:
        timenow = datetime.datetime.now()
        timelist.append(timenow)
        msg = data.decode('utf-8')
        windowlist.append(msg)
        if len(windowlist)<10:
            continue
            #10개 모으기
        else:
            df = pd.DataFrame.from_dict({'sensor':windowlist})
            input_data=np.array([[(timelist[9]-timelist[0]).total_seconds()
                     , df[df["sensor"]=='LR0 On'].count()["sensor"]
                     , df[df["sensor"]=='LR1 On'].count()["sensor"]
                     , df[df["sensor"]=='LR2 On'].count()["sensor"]
                     , df[df["sensor"]=='LR3 On'].count()["sensor"]
                     , df[df["sensor"]=='LR4 On'].count()["sensor"]
                     , df[df["sensor"]=='LR5 On'].count()["sensor"]
                     , df[df["sensor"]=='LR6 On'].count()["sensor"]
                     , df[df["sensor"]=='LR7 On'].count()["sensor"]
                     , df[df["sensor"]=='LR8 On'].count()["sensor"]
                     , df[df["sensor"]=='LR9 On'].count()["sensor"]
                     , df[df["sensor"]=='LR10 On'].count()["sensor"]
                     , df[df["sensor"]=='LR11 On'].count()["sensor"]
                     , df[df["sensor"]=='LR12 On'].count()["sensor"]
                     , df[df["sensor"]=='LR13 On'].count()["sensor"]
                     , df[df["sensor"]=='LR14 On'].count()["sensor"]
                     , df[df["sensor"]=='LP0 On'].count()["sensor"]
                     , df[df["sensor"]=='LP1 On'].count()["sensor"]
                     , df[df["sensor"]=='LP2 On'].count()["sensor"]
                     , df[df["sensor"]=='LD On'].count()["sensor"]
                     , df[df["sensor"]=='BeR0 On'].count()["sensor"]
                     , df[df["sensor"]=='BeR1 On'].count()["sensor"]
                     , df[df["sensor"]=='BeR2 On'].count()["sensor"]
                     , df[df["sensor"]=='BeR3 On'].count()["sensor"]
                     , df[df["sensor"]=='BeR4 On'].count()["sensor"]
                     , df[df["sensor"]=='BeR5 On'].count()["sensor"]
                     , df[df["sensor"]=='BeR6 On'].count()["sensor"]
                     , df[df["sensor"]=='BeR7 On'].count()["sensor"]
                     , df[df["sensor"]=='BeR8 On'].count()["sensor"]
                     , df[df["sensor"]=='BeP0 On'].count()["sensor"]
                     , df[df["sensor"]=='BeP1 On'].count()["sensor"]
                     , df[df["sensor"]=='BeP2 On'].count()["sensor"]
                     , df[df["sensor"]=='BeD On'].count()["sensor"] 
                     , df[df["sensor"]=='BathR0 On'].count()["sensor"]
                     , df[df["sensor"]=='BathR1 On'].count()["sensor"]
                     , df[df["sensor"]=='BathR2 On'].count()["sensor"]
                     , df[df["sensor"]=='BathP0 On'].count()["sensor"]
                     , df[df["sensor"]=='BathD On'].count()["sensor"]]])
            
            del windowlist[0]
            del timelist[0]
            predictions = loaded_model.predict(input_data)
            prof = round(float(np.amax(loaded_model.predict_proba(input_data))),3)
            cur = datetime.datetime.now()
            row = [cur, predictions[0], prof]
            #f = open('C:/Users/Shin/Documents/testbeddata/result/RF_SC_result.csv', 'a', encoding='utf-8')
            #wr = csv.writer(f)
            #wr.writerow(row)
            #f.close()
            print('State:'+' '+str(cur.strftime('%m-%d-%y %H:%M:%S'))+' '+blankreturn(predictions[0]), end="\r", flush=True)